<div style="background: navy; color: white; padding: 20px">

# $\S 6$: The rank 2 MMD as an improvement over the rank 1 MMD

## Z Issa Apr 2023

In this notebook, we show how using the $\mathrm{MMD}_2$ as a metric for market regime detection problems can lead to improved performance than by simply using the $\mathrm{MMD}_1$. 

### Background.

<div style="background: navy; color: white; padding: 20px">


### Imports

In [ ]:
from src.generators.config import ModelConfig
from src.testing import TestConfig
from src.testing.discriminators import GeneralMMDDetector
from src.testing.discriminators.config import ProcessorConfig
from src.testing.experiment_functions.mmd_test_functions import get_beliefs_from_config
from src.utils.Config import Config
from src.utils.auxiliary_classes.PathTransformer import PathTransformer, PathTransformerConfig
from src.utils.auxiliary_classes.RegimePartitioner import RegimePartitionerConfig

<div style="background: navy; color: white; padding: 20px">

    
### 1. Set configurations

In [ ]:
test_config             = TestConfig()
path_transformer_config = PathTransformerConfig()
regime_config           = RegimePartitionerConfig()

year_mesh         = 252
time              = 1/12
dim               = 5

mmd2_model_config = ModelConfig()
mmd2_model_config.override_args({
    "year_mesh": year_mesh
})

path_transformer_config.set_transformations({
    "standardise_path_transform":    (True,  0, {"s_type": "initial"}),
    "time_normalisation_transform":  (True,  0, {}),
    "difference_transform":          (False, 0, {}),
    "squared_log_returns_transform": (False, 0, {}),
    "ewma_volatility_transform"    : (False, 0, {"lambd": 0.5}),
    "cumulant_transform":            (True, 2, {}),
    "increment_transform":           (False, 2, {}),
    "lead_lag_transform":            (False, 3, {}),
    "invisibility_transform":        (False, 4, {}),
})

path_transformer_config.compute_pathwise_signature_transform = False
path_transformer_config.signature_order = 8

test_config.override_args({
    "n_steps"          : int(time*year_mesh),
    "n_paths"          : 10,
    "offset"           : 0,
    "weight_factor"    : 1,
    "belief_models"    : ["gbm"],
    "model_pair_names" : ["gbm", "rBergomi"],
    "belief_params"    : [[[0., 0.2] for _ in range(dim)]],
    "model_pair_params": [[[0., 0.2] for _ in range(dim)], [[0.1, 0.1, -0.7, 0.3] for _ in range(dim)]],
    "path_bank_size"   : 100000
})



mmd1_processor_config = ProcessorConfig()
mmd2_processor_config = ProcessorConfig()

mmd1_processor_config.override_args({
    "generalmmddetector_kwargs": Config(**{
        "normalise": False,
        "n_tests": 512,
        "n_evaluations": 1,
        "metric_kwargs": Config(**{
            "kernel_type": "rbf",
            "metric_type": "mmd",
            "sigmas": [sigma0],  
            "dyadic_orders": [0],  
            "lambd": 1
        }),
        "evaluator_kwargs": Config(**{
            "pct_ignore": 0.1
        })
    })
})

mmd2_processor_config.override_args({
    "generalmmddetector_kwargs": Config(**{
        "normalise": False,
        "n_tests": 512,
        "n_evaluations": 1,
        "metric_kwargs": Config(**{
            "kernel_type": "rbf",
            "metric_type": "mmd",
            "sigmas": [sigma0, sigma1],  
            "dyadic_orders": [0, 0],  
            "lambd": 1
        }),
        "evaluator_kwargs": Config(**{
            "pct_ignore": 0.1
        })
    })
})
        
beliefs, belief_details, model_pairs = get_beliefs_from_config(mmd2_test_config, mmd2_model_config)
path_transformer = PathTransformer(path_transformer_config)

### 2. Initialize detector objects

In [ ]:
mmd1_detector = GeneralMMDDetector(
    beliefs          = beliefs, 
    path_details     = belief_details,
    path_transformer = path_transformer,
    processor_config = mmd1_processor_config,
    test_config      = mmd2_test_config
)

mmd2_detector = GeneralMMDDetector(
    beliefs          = beliefs, 
    path_details     = belief_details,
    path_transformer = path_transformer,
    processor_config = mmd2_processor_config,
    test_config      = mmd2_test_config
)

### 3. Build regime-changed path

